In [115]:
import MeCab as mc
import re
import requests
from bs4 import BeautifulSoup
import sqlite3
import oseti

from gensim.models import Word2Vec

In [116]:
o = oseti.Analyzer()

In [33]:
conn = None

# データベースに接続する
def connect():
    global conn
    conn = sqlite3.connect("./anime.db")
    
# コネクションを断つ
def close():
    conn.close()
    
# テーブル作成
def create_table_Url():
    # DROP=消す.
    conn.execute("DROP TABLE IF EXISTS Url")
    conn.execute("""CREATE TABLE Url (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            url TEXT
    )""")

def create_table_Reviews():
    # DROP=消す.
    conn.execute("DROP TABLE IF EXISTS Reviews")
    conn.execute("""CREATE TABLE Reviews (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            title TEXT,
            review TEXT,
            rate FLOAT
    )""")
    
def load_Url(url):
    conn.execute("INSERT INTO Url (url) VALUES (?)", (url,))
    conn.commit()
    
def load_Reviews(title, review, rate):
    conn.execute("INSERT INTO Reviews (title,review,rate) VALUES (?,?,?)", (title,review,rate))
    conn.commit()

In [ ]:
url = 'https://sakuhindb.com'
next_url = "/anime/anime.html"

In [111]:
try:
    connect()
    create_table_Url()
finally:
    close()

for i in range(30):
    next_url = next_page(url,next_url,i)
    print(str(i+1)+"回目")

1回目
2回目
3回目
4回目
5回目
6回目
7回目
8回目
9回目
10回目
11回目
12回目
13回目
14回目
15回目
16回目
17回目
18回目
19回目
20回目
21回目
22回目
23回目
24回目
25回目
26回目
27回目
28回目
29回目
30回目


In [29]:
def next_page(url,next_url,i):
    try:
        connect()
        r = requests.get(url+next_url)
        html_soup = BeautifulSoup(r.text, 'html.parser')
        new_url_all = [r["href"] for r in html_soup.find("td", class_ ="article container").find_all("a")]
        next_url = new_url_all[len(new_url_all)-1]
        if i != 0:
            del new_url_all[0:2]
            del new_url_all[len(new_url_all)-2:]
        else:
            del new_url_all[len(new_url_all)-1]
        for new_url in new_url_all:
            load_Url(new_url)
        return next_url
    finally:
        close()

In [39]:
def get_url():
    try:
        connect()
        cursor = conn.cursor()
        res = cursor.execute("SELECT url FROM Url")
        res = cursor.fetchall()
        return res
    finally:
        close()

In [137]:
def l_R(url,g_u):
    r = requests.get(url+g_u)
    res_soup = BeautifulSoup(r.content, 'html.parser')
    f_table = res_soup.find("td", class_="padding_cell").find_all("div", itemprop="reviewBody")
    text_all=""
    for text in f_table:
        text_all += text.text

    rate_all=[]
    for rate in o.analyze(text_all):
        if int(rate) != 0:
            rate_all.append(rate)
    
    if len(rate_all)==0:
        rate = 0.0
    else:
        rate=sum(rate_all)/len(rate_all)
    title = res_soup.find("span", itemprop="name").text
    try:
        connect()
        load_Reviews(title, text_all, rate)
    finally:
        close()

In [138]:
try:
    connect()
    create_table_Reviews()
finally:
    close()
for g_u in get_url():
    l_R(url,g_u[0])

In [ ]:
def mecab_analysis(texts):
    t = mc.Tagger("")
    t.parse('')
    output = []
    node =  t.parseToNode(texts)  
    while node:
        if node.surface != "":  # ヘッダとフッタを除外
            word_type = node.feature.split(",")[0]
            if word_type in ['名詞']:
                output.append(node.surface)
        node = node.next
        if node is None:
            break
    return output

In [ ]:
def calc_similarity(text1,text2):
    sim = model.similarity(text1,text2)
    return sim

In [ ]:
revs = []
for rev in get_review(1):
    revs.append(mecab_analysis(rev[0]))

In [ ]:
model = Word2Vec(revs, size =100, window=5, min_count=1, workers=4)
model.save("word2vec.model")

calc_similarity("BBQ","テリヤキ")

similar_word = model.most_similar(positive="BBQ")
similar_word